<a href="https://colab.research.google.com/github/willystw/fastai-learning/blob/bear-multi-classification/movie-recommender/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *

In [3]:
from fastai.collab import *
from fastai.tabular.all import *

In [15]:
path = untar_data('https://files.grouplens.org/datasets/movielens/ml-latest-small.zip')

In [16]:
path.ls()

(#5) [Path('/root/.fastai/data/ml-latest-small/ratings.csv'),Path('/root/.fastai/data/ml-latest-small/movies.csv'),Path('/root/.fastai/data/ml-latest-small/links.csv'),Path('/root/.fastai/data/ml-latest-small/README.txt'),Path('/root/.fastai/data/ml-latest-small/tags.csv')]

In [17]:
ratings = pd.read_csv(path/'ratings.csv')
movies = pd.read_csv(path/'movies.csv',usecols=(0,1))



In [18]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [19]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [20]:
tags = pd.read_csv(path/'tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [1]:
#Crash
#ratings = ratings.merge(tags)
#ratings.head()

NameError: ignored

In [21]:
ratings = ratings.merge(movies)
ratings.head()

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story (1995)
1,5,1,4.0,847434962,Toy Story (1995)
2,7,1,4.5,1106635946,Toy Story (1995)
3,15,1,2.5,1510577970,Toy Story (1995)
4,17,1,4.5,1305696483,Toy Story (1995)


In [23]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,userId,title,rating
0,195,Die Hard 2 (1990),3.0
1,50,Little Miss Sunshine (2006),3.0
2,525,Daredevil (2003),2.0
3,1,Forrest Gump (1994),4.0
4,380,Austin Powers: The Spy Who Shagged Me (1999),3.0
5,414,Fantastic Mr. Fox (2009),4.0
6,448,Monty Python Live at the Hollywood Bowl (1982),4.0
7,7,K-PAX (2001),3.0
8,181,"American President, The (1995)",5.0
9,243,Crimson Tide (1995),5.0


In [24]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))

In [26]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.364976,0.755225,00:10
1,0.448380,0.759124,00:10
2,0.334750,0.750739,00:10
3,0.235788,0.749146,00:10
4,0.139591,0.751357,00:10


In [27]:
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.827297,0.816885,00:13
1,0.758520,0.758206,00:13
2,0.707886,0.739721,00:13
3,0.629852,0.733222,00:13
4,0.546671,0.745621,00:13


In [30]:
num_users = len(dls.classes['userId'])
num_movies = len(dls.classes['title'])

num_users, num_movies

(611, 9720)

In [31]:
class DotProduct(Module):
  def __init__(self, n_movies, n_users, n_factors, y_range=(0,5.5)):
    self.users_factors = Embedding(n_users, n_factors)
    self.movies_factors = Embedding(n_movies, n_factors)
    self.y_range = y_range

  def forward(self, x):
    users = self.users_factors(x[:,0])    
    movies = self.movies_factors(x[:,1])
    return sigmoid_range((users * movies).sum(dim=1), *self.y_range)

In [32]:
class DotProductWithBias(Module):
  def __init__(self, n_movies, n_users, n_factors, y_range=(0,5.5)):
    self.users_factors = Embedding(n_users, n_factors)
    self.users_bias = Embedding(n_users, 1)
    self.movies_factors = Embedding(n_movies, n_factors)
    self.movies_bias = Embedding(n_movies, 1)
    self.y_range = y_range

  def forward(self, x):
    users = self.users_factors(x[:,0])    
    movies = self.movies_factors(x[:,1])

    result = (users*movies).sum(dim=1, keepdim=True)
    result += self.users_bias(x[:,0]) + self.movies_bias(x[:,1])
    return sigmoid_range(result, *self.y_range)

In [33]:
model = DotProduct(num_movies, num_users, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.910968,0.898482,00:09
1,0.740779,0.798156,00:09
2,0.506790,0.789077,00:09
3,0.303921,0.795678,00:09
4,0.229063,0.799937,00:09


In [34]:
model_bias = DotProductWithBias(num_movies, num_users, 50)
learn = Learner(dls, model_bias, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.828291,0.801374,00:10
1,0.705147,0.727337,00:10
2,0.434847,0.744625,00:13
3,0.236306,0.763024,00:10
4,0.139194,0.767161,00:10


In [35]:
model_bias = DotProductWithBias(num_movies, num_users, 50)
learn = Learner(dls, model_bias, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.783097,0.800990,00:10
1,0.696954,0.737766,00:10
2,0.559897,0.714609,00:10
3,0.393810,0.708439,00:10
4,0.300678,0.710253,00:10


In [36]:
learn.model

DotProductWithBias(
  (users_factors): Embedding(611, 50)
  (users_bias): Embedding(611, 1)
  (movies_factors): Embedding(9720, 50)
  (movies_bias): Embedding(9720, 1)
)

In [37]:
movie_bias = learn.model.movies_bias.weight.squeeze()

In [38]:
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Shawshank Redemption, The (1994)',
 'Forrest Gump (1994)',
 'Eternal Sunshine of the Spotless Mind (2004)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Up (2009)']

In [39]:
embs = get_emb_sz(dls)
embs

[(611, 58), (9720, 274)]

In [40]:
class CollabNN(Module):
  def __init__(self, user_size, item_size, y_range=(0,5.5), n_act = 100):
    self.user_factors = Embedding(*user_size)
    self.item_factors = Embedding(*item_size)
    self.y_range = y_range
    self.layers = nn.Sequential(
        nn.Linear(user_size[1] + item_size[1], n_act),
        nn.ReLU(),
        nn.Linear(n_act, 1)
    )
  def forward(self, x):
    embs = self.user_factors(x[:,0]), self.item_factors(x[:,1])
    x = self.layers(torch.cat(embs, dim=1))
    return sigmoid_range(x, *self.y_range)

In [41]:
nn_model = CollabNN(*embs)
learn = Learner(dls, nn_model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.827410,0.800621,00:12
1,0.742702,0.773231,00:12
2,0.676327,0.731303,00:12
3,0.622190,0.740300,00:12
4,0.543628,0.752794,00:12
